# 4. Deep Learning

## 4.1 Tensorflow & Keras

### Tensorflow Basic
* tf.constant
* tf.Session
* tf.Variable
* tf.placeholder
* tf.graph
* life cycle

In [1]:
import tensorflow as tf

## 상수
* tf.constant()로 생성하는 상수는 초기화 없이 사용이 가능하다. 
    * 하지만 값의 변경이 되지 않는다
* 값은 세션에 저장된다.


In [2]:
hello = tf.constant('Hello World!')
print(hello)
#dir(hello)

Tensor("Const:0", shape=(), dtype=string)


In [3]:
a = tf.constant(10)
b = tf.constant(20)
c = a + b
print(c)

Tensor("add:0", shape=(), dtype=int32)


In [4]:
cc = tf.add(a, b)
print(cc)

Tensor("Add_1:0", shape=(), dtype=int32)


## Session
### `tf.Session()`
* 노드들로 구성된 그래프를 실행하려면 Session이 필요하다.
* 그래프를 CPU 또는 GPU에 로드하고 실행한다.
    * `run()`
* `sess = tf.Session()`으로 생성할 수 있고 `sess.run()`으로 텐서를 실행할 수 있다.
* 모든 tensor들은 `t.eval()` 메서드를 가지고 있는데, 이것은 `tf.get_default_session.run(t)` 한것과 같다.
    * 또는 `t.eval(session=sess)`로 세션을 지정할 수 도 있다.
* `with tf.Session() as sess :` 구문으로 블록내의 default session을 지정할 수 있다.
* `with sess.as_default()` 함수로 default session을 지정할 수도 있다.
* `sess.close()` 함수로 자원을 반납해야 한다.

### `tf.InteractiveSession()`
* 자동으로 전역으로 사용할 default session으로 등록하게 된다.
* with 구문이 필요없다.

In [5]:
sess = tf.Session()
#dir(sess)

In [6]:
sess.run(c)

30

In [7]:
sess.run(cc)

30

In [8]:
sess.run(a)

10

## 변수 , `tf.Variable()`
* 값을 변경할 수 있는 변수
* `tf.Variable(val)`
* 보통 학습 모델에 의해 Wegith 값을 보관할 목적으로 사용
    * 학습 과정에서 값이 변경된다.
    * 초기 값을 주기 애매 할 때는 랜덤 값등을 사용 할 수 있다.
    * `tf.Variable(tf.random_normal([n, m])`
* 사용하기 전에 초기화를 반드시 수행해야 한다.
    *  sess.run(t.initializer) 
    * t.initializer.run()
        * default session 이 있는 경우
* 여러 변수를 한꺼번에 초기화 할때
    * `sess.run(tf.global_variables_initializer())`
    * `tf.global_variable_initialzer.run()` 
        * default session 있는 경우

In [9]:
x = tf.Variable(3)
y = tf.Variable(4)
z = x * y
print(x, y, z)

Instructions for updating:
Colocations handled automatically by placer.
<tf.Variable 'Variable:0' shape=() dtype=int32_ref> <tf.Variable 'Variable_1:0' shape=() dtype=int32_ref> Tensor("mul:0", shape=(), dtype=int32)


In [10]:
print(tf.get_default_session())

None


In [11]:
print(tf.global_variables_initializer())

name: "init"
op: "NoOp"
input: "^Variable/Assign"
input: "^Variable_1/Assign"



In [12]:
sess.run(tf.global_variables_initializer())

In [13]:
sess.run([x,y,z])

[3, 4, 12]

In [14]:
z.eval(session=sess)

12

In [15]:
with sess.as_default():
    print(z.eval())

12


In [16]:
sess2 = tf.InteractiveSession()

In [17]:
x.initializer.run()
y.initializer.run()
z.eval()

12

## PlaceHolder
* 매개변수, 식은 정해져 있지만 값은 실행할때 마다 달라져야 하는 것들
* `tf.placeholder(tf.float32, [None, 2])`
* 실제로 값을 전달하는 곳은 `sess.run()`에서 `feed_dict={A:a, B:b}` 형식으로 전달

In [18]:
tf.reset_default_graph()
x_data = [1,2,3,4,5]
X = tf.placeholder(tf.int32)
adder = tf.constant(10)
y = tf.Variable(0)
y = X + adder
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(y, feed_dict={X:x_data}))


[11 12 13 14 15]


### 그래프, graph
* 상수, 변수 등의 텐서를 생성하면 이것은 하나의 계산 그래프에 추가된다.
* 각 텐서의 `t.graph` 속성에 지정된다
* 새로운 그래프를 만드려면 `tf.Graph()` 함수가 있다.
* 새로운 그래프를 사용하려면 `with graph.as_default():`로 블럭 설정을 해서 사용할 수 있다.
* 텐서를 생성하는것과 같은 코드를 반복해서 실행하면 그래프에 중복된 텐서가 텐서 노드가 포함되기 때문에 오류가 발생
    * 파이썬 프로세스 또는 커널을 재 실행 또는
    * `tf.reset_default_graph()`를 실행
 

## 노드의 Life Cycle
* 종속성 있는 노드는 평가 시점에 모두 평가가 같이 실행된다.
* 종속성있는 노드를 2번 평가되면 그 과정의 노드들도 2번 평가된다.
* 한번에 하고 싶으면 리스트로 만들어서 한번에 평가하라.

### 세션 닫고 재 실행
* 세션을 생성하고 닫은 후 다시 실행하면 값을 항상 처음으로 돌아 간다.

In [19]:
w = tf.constant(2)
x = tf.Variable(3)
y =  w + x

sess = tf.Session()
sess.run(tf.global_variables_initializer())

y = y *2 
z = y + 1

print(sess.run(w), sess.run(x), sess.run(y))
print(sess.run(z))

sess.close()

2 3 10
11


### 세션 유지 재 실행
* 하지만 세션을 닫지 않고 실행만 다시 하면 값을 유지된다.

In [20]:
w = tf.constant(2)
x = tf.Variable(3)
y =  w + x

sess = tf.Session()
sess.run(tf.global_variables_initializer())


In [21]:
y = y *2 
z = y + 1

print(sess.run(w), sess.run(x), sess.run(y))
print(sess.run(z))


2 3 10
11


### 개별 노드 실행
* 세션을 실행할때 개별적으로 실행하면 종속성있는 모든 노드가 매번 실행 된다.
* 아래의 경우 z의 값이 y의 값을 2번 변경하게 한다.

In [22]:
w = tf.constant(2)
x = tf.Variable(3)
y = tf.Variable(0)
y =  tf.assign(y, y + x)
z = y + 1
sess = tf.Session()
sess.run(tf.global_variables_initializer())


print(sess.run(w), sess.run(x), sess.run(y), sess.run(z))
sess.close()

2 3 3 7


### 여러 노드 한번에 실행
* 세션에 노드를 리스트로 묶어서 실행하면 한번에 평가된다.
* 아래의 코드의 z는 y를 두번 평가하게 하지 않는다.

In [23]:
w = tf.constant(2)
x = tf.Variable(3)
y = tf.Variable(0)
y =  tf.assign(y, y + x)
z = y + 1
sess = tf.Session()
sess.run(tf.global_variables_initializer())


print(sess.run([w,x,y,z]))

sess.close()

[2, 3, 3, 4]


### Tensorflow를 이용한 LinerRegression

In [24]:
import tensorflow as tf
import numpy as np

x_data = np.arange(0,10)
y_data = x_data * 2 + 10

### 변수, placeholder 초기화

In [25]:
# -1.0 ~ 1.0 균등분포를 갖는 랜덤 값
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0)) 
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

# 입력을 위한 placeholder
X = tf.placeholder(tf.float32, name="X")
Y = tf.placeholder(tf.float32, name="Y")
print(X)
print(Y)

# 가설 생성
hypothesis = W * X + b

Tensor("X:0", dtype=float32)
Tensor("Y:0", dtype=float32)


### 손실함수와 손실 최소화 옵티마이저 생성

In [26]:
# 손실 함수를 작성합니다.
cost = tf.reduce_mean(tf.square(hypothesis - Y))
# 텐서플로우에 기본적으로 포함되어 있는 함수를 이용해 경사 하강법 최적화를 수행합니다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
# 비용을 최소화 하는 것이 최종 목표
train_op = optimizer.minimize(cost)

Instructions for updating:
Use tf.cast instead.


### 훈련 실행 및 테스트


In [27]:
epochs = 500
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # 최적화를 100번 수행합니다.
    for step in range(1, epochs+1):
        # sess.run 을 통해 train_op 와 cost 그래프를 계산합니다.
        # 이 때, 가설 수식에 넣어야 할 실제값을 feed_dict 을 통해 전달합니다.
        _, cost_val = sess.run([train_op, cost], feed_dict={X: x_data, Y: y_data})
        if step %20 == 0:
            print(step, cost_val, sess.run(W), sess.run(b))
  
# 최적화가 완료된 모델에 테스트 값을 넣고 결과가 잘 나오는지 확인해봅니다.
    print("\n=== Test ===")
    print("X: 5, Y:%f, Y':%f"%(5*2+10, sess.run(hypothesis, feed_dict={X: 5})))
    print("X: 2.5, Y:%f, Y':%f"%(2.5*2+10, sess.run(hypothesis, feed_dict={X: 2.5})))

20 17.247007 [3.2238588] [2.325688]
40 13.750941 [3.0928001] [3.1475039]
60 10.963552 [2.975776] [3.8813136]
80 8.7411785 [2.8712833] [4.536543]
100 6.9692917 [2.7779808] [5.121606]
120 5.5565767 [2.6946692] [5.6440167]
140 4.4302287 [2.6202796] [6.1104836]
160 3.5321968 [2.5538561] [6.526998]
180 2.8162024 [2.4945457] [6.898909]
200 2.2453432 [2.4415865] [7.2309937]
220 1.790201 [2.3942988] [7.527516]
240 1.4273177 [2.3520746] [7.7922854]
260 1.1379938 [2.3143723] [8.028702]
280 0.90731496 [2.2807071] [8.239801]
300 0.7233993 [2.2506475] [8.428293]
320 0.57676095 [2.2238064] [8.596603]
340 0.4598493 [2.1998398] [8.746887]
360 0.36663467 [2.1784396] [8.881079]
380 0.2923161 [2.1593313] [9.0009]
400 0.23306184 [2.142269] [9.107891]
420 0.1858184 [2.1270337] [9.203424]
440 0.1481525 [2.1134303] [9.288726]
460 0.118121244 [2.1012836] [9.364893]
480 0.09417732 [2.0904372] [9.432905]
500 0.07508698 [2.0807526] [9.493634]

=== Test ===
X: 5, Y:20.000000, Y':19.897396
X: 2.5, Y:15.000000, Y':

### Keras를 이용한 Linear Regress

In [1]:
import keras
import numpy as np

x_data = np.arange(0,10)
y_data = x_data * 2 + 10

Using TensorFlow backend.


#### 네트워크 생성
* kears.models.Sequential : Linear stack network
* keras.layers.Dense : Layer, 완전 연결층(밀집연결층)
    * 생성자에 들어 가는 숫자는 출력 노드의 갯수이다.
    * 필요에 따라 activation 함수를 선택
    * 첫 입력 층인경우 `input_shape=(r,c)` 형태의 파라미터가 필요하다.

In [2]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(1, input_shape=(1,)))

Instructions for updating:
Colocations handled automatically by placer.


#### compile() : 다음 3가지를 자동으로 추가해 준다.
    * cost function
    * optimizer
    * 평가(정확도) 지표

In [3]:
#model.compile('SGD', 'mse')
model.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])
#sgd = keras.optimizers.SGD(lr=0.01)
#model.compile(optimizer=sgd, loss='mse', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


### 훈련 및 평가

In [6]:
model.fit(x_data, y_data, epochs=500, verbose=0)
weights = model.layers[0].get_weights()
print("W: %f, b: %f"%(weights[0], weights[1]))
print("X: 5, Y:%f, Y':%f"%(5*2+10,  model.predict([5])))
print("X: 2.5, Y:%f, Y':%f"%(12.5*2+10,  model.predict([2.5])))

W: 2.000301, b: 9.998110
X: 5, Y:20.000000, Y':19.999617
X: 2.5, Y:35.000000, Y':14.998863


### Tensorflow를 이용한 Softmax

In [5]:
# 털과 날개가 있는지 없는지에 따라, 포유류인지 조류인지 분류하는 신경망 모델을 만들어봅니다.
import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))
L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)
model = tf.nn.softmax(L)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
epochs = 200
for step in range(1, epochs+1):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    if (step + 1) % 20 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))


20 0.96481395
40 0.9598415
60 0.95506054
80 0.9503887
100 0.9458931
120 0.9415844
140 0.9373496
160 0.93331534
180 0.92936856
200 0.9255397
예측값: [2 2 2 2 2 2]
실제값: [0 1 2 0 0 2]
정확도: 33.33


### Keras를 이용한 Softmax 예제

In [7]:
import keras.models as models
import keras.layers as layers
import numpy as np

x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

model = models.Sequential()
model.add(layers.Dense(3, activation='softmax', input_shape=(2,)))
model.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])

model.fit(x_data, y_data, epochs=200, verbose=0)

pred = np.argmax(model.predict(x_data), axis=1)
target = np.argmax(y_data, axis=1)
print('예측값:', pred)
print('실제값:', target)
print('정확도: %.2f'% (np.sum(pred == target)/target.shape[0]*100))


예측값: [0 2 2 0 0 1]
실제값: [0 1 2 0 0 2]
정확도: 66.67
